# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath): 
# join the file path and roots with the subdirectories using glob
    file_path_list += glob.glob(os.path.join(root,'*'))
file_path_list = list(set(file_path_list))
file_path_list.sort()
for i in file_path_list:
    print(i)

/home/workspace
/home/workspace/event_data/2018-11-01-events.csv
/home/workspace/event_data/2018-11-02-events.csv
/home/workspace/event_data/2018-11-03-events.csv
/home/workspace/event_data/2018-11-04-events.csv
/home/workspace/event_data/2018-11-05-events.csv
/home/workspace/event_data/2018-11-06-events.csv
/home/workspace/event_data/2018-11-07-events.csv
/home/workspace/event_data/2018-11-08-events.csv
/home/workspace/event_data/2018-11-09-events.csv
/home/workspace/event_data/2018-11-10-events.csv
/home/workspace/event_data/2018-11-11-events.csv
/home/workspace/event_data/2018-11-12-events.csv
/home/workspace/event_data/2018-11-13-events.csv
/home/workspace/event_data/2018-11-14-events.csv
/home/workspace/event_data/2018-11-15-events.csv
/home/workspace/event_data/2018-11-16-events.csv
/home/workspace/event_data/2018-11-17-events.csv
/home/workspace/event_data/2018-11-18-events.csv
/home/workspace/event_data/2018-11-19-events.csv
/home/workspace/event_data/2018-11-20-events.csv
/hom

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 

# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object
        csvreader = csv.reader(csvfile) 
        next(csvreader)
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full\
# csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7],
                         row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#create a keysapace named 'music_keyspace'
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set the keyspace
try:
    session.set_keyspace('music_keyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Creating a table that can provide the artist, song title and song's length 
## in the music app history that was heard during sessionId = 338 
## and itemInSession = 4 ...as an example.

query_drop = "DROP TABLE IF EXISTS session_table"

try:
    session.execute(query_drop)
except Exception as e:
    print(e)
# create table query. The primary key is sessionId and itemInSession because \
# those are the values in the WHERE clause, and together represent a unique item.
query = "CREATE TABLE IF NOT EXISTS session_table "
query = query + "(sessionId int, itemInSession int, artist text, song text, \
                  length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    #insert csv row values into the table
    for line in csvreader:
        query = "INSERT INTO session_table (sessionId, itemInSession, \
                 artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
# a query based off of the query goal for the table
query1 = "SELECT artist, song, length FROM session_table WHERE sessionId=338 AND itemInSession=4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


In [11]:
## Creating a table that can provide name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query_drop = "DROP TABLE IF EXISTS user_session"

try:
    session.execute(query_drop)
except Exception as e:
    print(e)
# create table query. The primary key is (userId, sessionid), itemInSession
# userId and sessionid are the partition key because we do not want them 
# split across nodes. itemInSession is a clustering column

query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, \
                  song text, firstName text, lastName text, \
                  PRIMARY KEY ((userId, sessionid), itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, \
                  song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],
                                line[1], line[4]))
                    

In [13]:
# a query based off of the query goal for the table
query2 = "SELECT artist, song, firstName, lastName from user_session WHERE userId = 10 AND sessionid = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    #print(row)
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## Creating a table that can provide user name (first and last) in 
## my music app history who listened to the song 'All Hands Against His Own'

query_drop = "DROP TABLE IF EXISTS listened_to_song"

try:
    session.execute(query_drop)
except Exception as e:
    print(e)

# create table with Primary Key of song, and userId. This table is designed
# to only include one combination of firstName, lastName with each song.
query = "CREATE TABLE IF NOT EXISTS listened_to_song "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId));"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    #insert csv row values into the table
    for line in csvreader:
        query = "INSERT INTO listened_to_song (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s) IF NOT EXISTS"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))



In [16]:
query3 = "SELECT firstName, lastName FROM listened_to_song WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [17]:
#drop the tables

query = "drop table session_table"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table listened_to_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
# shut down the session and cluster
session.shutdown()
cluster.shutdown()